In [1]:
'''

整體功能描述

'''

'\n\n整體功能描述\n\n'

In [2]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [3]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,StickerSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName) as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

def detect_json_array_to_linebot_message_array(jsonArray):

    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

In [4]:
'''

handler處理關注消息

用戶關注時，讀取 素材 -> 關注 -> reply.json

將其轉換成可寄發的消息，傳回給Line

'''

# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):

    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("./users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
    
    
    # 呼叫個人用戶名，與用戶第一次接觸打招呼
    user_profile = line_bot_api.get_profile(event.source.user_id)
    hello = [{
              "type": "image",
              "originalContentUrl": "https://"+server_url+"/material/%E9%97%9C%E6%B3%A8/logo_push_removebg.png",
              "previewImageUrl": "https://"+server_url+"/material/%E9%97%9C%E6%B3%A8/logo_push_removebg.png"
            },
            {
              "type": "text",
              "text": "哈囉!" + str(user_profile.display_name) + \
                      "\n歡迎您使用「履視步爽」\n目前我們的服務完全免費"+u"\U0001f600"
            },
            {
              "type": "text",
              "text": "想要使用我們的服務，請點選下方「屢視步爽」，開啟圖文選單選取你想使用的功能，並讓我們了解你的喜好，以便提供給您最適合的商品推薦，還請耐心回答，謝謝!"
            },
            {
              "type": "sticker",
              "packageId": "11539",
              "stickerId": "52114110"
            }]

    result_message_array = detect_json_array_to_linebot_message_array(hello)
    
    #綁定 rich_menu
    linkRichMenuId = open("material/rich_menu_4/rich_menu_id", 'r').read()
    line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)

    # 消息發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )

In [5]:
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from jieba import cut

imgMsgkey = ''

# 使用joblib.load()呼叫模型
vec = CountVectorizer()
clf2 = joblib.load('./models/clf_4.pkl')
vec = joblib.load('./models/vec_4.pkl')
clf_dispution = joblib.load('./models/clf_dispution.pkl')
vec_dispution = joblib.load('./models/vec_dispution.pkl')

/root/miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/root/miniconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.21.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/root/miniconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.21.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:

    
def contentcut_label(s):
    label = ["換貨", "退貨", "賣家拒絕退貨"]
    s = s.replace("\r", "").replace("\n", "")
    c = " ".join(cut(s))
    v = vec_dispution.transform([c])
    predict = label[clf_dispution.predict(v)[0]]

    if predict == "換貨":
        resultMsg = ('''依照消費者保護法規定，消費者均享有商品到貨七天鑑賞期之權益。
                 在期間內，若商品尺寸不合、寄錯商品、或者商品本身有瑕疵可以退換貨。
                 退換貨時商品必須保持全新狀態及完整包裝，及吊牌未拆、未下水、未使用、無汙損、無香水味與異味，須完整無缺的全數退回。
                 請參考以下退換貨規則詳情。
                 以下事項可能影響您的"換貨"權限：
                        (1) 在不影響您檢查商品情形下，若將商品吊牌拆除或因個人不當拆卸而產生人為因素之故障、毀損、刮傷、髒污、不完整導致無法銷售等，恕無法受理換貨。
                        (2) 商品一經吊牌拆除、損壞或遺失；不慎沾染彩妝、香水味；自行修改；下水洗滌；超過7天，即不接受任何理由換貨。另貼身衣物用品（嬰兒包屁衣、連身衣、嬰兒褲子、嬰兒吊帶褲等）及消耗品，除了新品瑕疵外，一經拆封後恕不受理換貨，敬請見諒。
                        (3) 退回商品必須完整；若退回商品本身或配件等不完整恕不受理換貨。''')
        resultMsg = [{
                       "type": "text",
                       "text": resultMsg.replace(" ", "")
                     },
                     {
                       "type": "image",
                       "originalContentUrl": "https://"+server_url+"/material/dispution/exchange.jpg",
                       "previewImageUrl": "https://"+server_url+"/material/dispution/exchange.jpg"
                     }
                     ]

    elif predict == "退貨":
        resultMsg = ('''依照消費者保護法規定，消費者均享有商品到貨七天鑑賞期之權益。
                 在期間內，若商品尺寸不合、寄錯商品、或者商品本身有瑕疵可以退換貨。
                 退換貨時商品必須保持全新狀態及完整包裝，及吊牌未拆、未下水、未使用、無汙損、無香水味與異味，須完整無缺的全數退回。
                 請參考以下退換貨規則詳情:
                     以下事項可能影響您的"退貨"權限：
                        (1) 在不影響您檢查商品情形下，若將商品吊牌拆除或因個人不當拆卸而產生人為因素之故障、毀損、刮傷、髒污、不完整導致無法銷售等，恕無法受理退貨。
                        (2) 商品一經吊牌拆除、損壞或遺失；不慎沾染彩妝、香水味；自行修改；下水洗滌；超過7天，即不接受任何理由退貨。另貼身衣物用品（嬰兒包屁衣、連身衣、嬰兒褲子、嬰兒吊帶褲等）及消耗品，除了新品瑕疵外，一經拆封後恕不受理退貨，敬請見諒。
                        (3) 退回商品（包含商品本身、外盒及所有配件、外盒包裝）必須完整；若退回商品本身或配件、外盒等不完整恕不受理退貨。
                        (4) 隨貨附贈之贈品若未退回，不接受退貨申請。''')
        resultMsg = [
                     {
                       "type": "text",
                       "text": resultMsg.replace(" ", "")
                     },
                     {
                       "type": "image",
                       "originalContentUrl": "https://"+server_url+"/material/dispution/return.jpg",
                       "previewImageUrl": "https://"+server_url+"/material/dispution/return.jpg"
                     }]

    elif predict == "賣家拒絕退貨":
        resultMsg = ('''消保法規定消費者可以七日內無條件退貨的交易方式，僅限於「郵購買賣」和「訪問買賣」二種交易型態。
                 所謂「郵購買賣」，依據消保法第 2 條第 10 款規定，僅限於企業經營者是以廣播、電視、電話、傳真、型錄、報紙、雜誌、網際網路、傳單或其他類似之方法，讓消費者未能檢視商品而與企業經營者所成立之買賣。
                 另外所謂「訪問買賣」，係指企業經營者未經邀約而在消費者之住居所或其他場所從事銷售，所成立之買賣。故一般到賣場或實體商店購物，消費者都沒有消保法有關七日內無條件退貨的權利。''')
        resultMsg = [{
                       "type": "text",
                       "text": resultMsg.replace(" ", "")
                     }
                     ,
                     {
                       "type": "image",
                       "originalContentUrl": "https://"+server_url+"/material/dispution/reject.jpg",
                       "previewImageUrl": "https://"+server_url+"/material/dispution/reject.jpg"
                     },
                     ]
#     print([predict, resultMsg])
    return [predict, resultMsg]

In [7]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

轉譯json後，將消息回傳給用戶

'''

# 引用套件
from linebot.models import (
    MessageEvent, TextMessage
)

imgMsgkey = ''


# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    global imgMsgkey
    print("imgMsgkey:", imgMsgkey)
    print("text:",event.message.text)
    msg = event.message.text
    if msg == '確認結果':
        # 讀取資料夾內.jpg檔案的數量
        pwd = '/root/darknet/result_img'
        imgWaitToTest = [i for i in os.listdir(pwd) if '.jpg' in i]
        
        # 如果測試照片還沒輸出，回覆使用者稍後確認
        if len(imgWaitToTest) == 0:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='測試中，請稍後再確認，謝謝。')
            )
        
        # 如果測試照片已經輸出，則複製照片到對外IP的資料夾內，等待發送給使用者
        elif len(imgWaitToTest) != 0:
            CopyFileWaitToPost(event.source.user_id, event.message.id)
            
            # 準備輸出多張照片的list
            result_message_array = []
            while True:
                # 檢查照片是否已經到到對外IP的資料夾內部
                pwd = '/root/linechatbot/material/user/'+event.source.user_id
                if os.path.exists(pwd):
                    imgWaitToPost = [i for i in os.listdir(pwd) if '.jpg' in i]
                if len(imgWaitToPost) !=0 :
                    for img in imgWaitToPost:
                        url = 'https://'+server_url+'/material/user/'+event.source.user_id+'/'+img
                        print(url)
                        String = """ImageSendMessage(
                                     original_content_url="{}",
                                     preview_image_url="{}"
                                 )""".format(url, url)
                        result_message_array.append(eval(String))
                    
                    fn = '/root/linechatbot/material/user/'+event.source.user_id+'/'+img
                    recommend_list = remove_bg(fn)
                    result_message_array = Picfindshoe(611, recommend_list)
                    
                    line_bot_api.reply_message(
                        event.reply_token,
                        result_message_array
                    )
                    break
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='請上傳照片')
            )
#     elif msg == '確認顏色結果':
#         result_message_array = findSituation2(l)
#         line_bot_api.reply_message(
#             event.reply_token,
#             result_message_array
#         )
    
    elif msg == '圖片找鞋':
        imgMsgkey = msg
        line_bot_api.reply_message(
            event.reply_token,
            template_message_dict.get(msg)
        )
    
    elif msg == "顏色找鞋":
        imgMsgkey = msg
        line_bot_api.reply_message(
            event.reply_token,
            template_message_dict.get(msg)
        )
    
    elif msg == '情境推薦':
        imgMsgkey = msg
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='請隨意輸入你等會要參加的活動或是事情，我們將推薦給您適合的鞋款。\n例如:我等會要去打籃球\n或是:我今天有個面試\n或是:我要去日本東京自由行')
        )
    
    elif imgMsgkey == '情境推薦':
        imgMsgkey = None
        
        def contentcut(s):
            s = s.replace("\r", "").replace("\n", "")
            return " ".join(cut(s))

        label = ["casual", "sport", "formal", "other"]

        c = contentcut(msg)
        v = vec.transform([c])
        global clf2
        l = label[clf2.predict(v)[0]]
        
        print("我們可以推薦", l, "類型的鞋子給您。")
        result_message_array = findSituation2(l, event.source.user_id)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        
    elif msg == "退貨諮詢":
        imgMsgkey = msg
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='請問您是需要退貨、換貨或有任何消費糾紛嗎?\n請告訴我們，你遇到的狀況。\n我們將給予您一些基本的建議與處理。')
        )
        
    elif imgMsgkey == "退貨諮詢":
        imgMsgkey = None
        result = contentcut_label(msg)
        predict = result[0]
        result_message_array = result[1]
        first_msg = {
              "type": "text",
              "text": "您的狀況，屬於「"+predict+"」，您可以參考以上資訊，再作進一步的處理\n客服專線：0987-XX6483"
            }
        result_message_array.insert(1, first_msg)
        result_message_array = detect_json_array_to_linebot_message_array(result_message_array)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        
    elif msg == "個人推薦":
        pass
    
    elif "我要" in msg:
        print('我要...')
    
    elif "加入收藏" in msg:
        print('加入收藏...')
    
    elif msg in os.listdir('./material/'):
        pass
        
    else:
        # 發送
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='想要使用我們的服務，請點選下方「屢視步爽」，開啟圖文選單選取你想使用的功能，並讓我們了解你的喜好，以便提供給您最適合的商品推薦，還請耐心回答，謝謝!"')
        )


In [8]:
from kafka import KafkaProducer
import sys

def sendkafka(rating, product_id, userid):
    producer = KafkaProducer(
        # 指定Kafka集群伺服器(["docker.compose.host.name:9092"])
        bootstrap_servers = ["kafka:9092"],
        value_serializer = lambda m: json.dumps(m).encode('ascii'),
    )
    try:
        import mysql.connector

        cursor = None
        conn = None
        try:
            # 建立連線 並指定 database
            conn = mysql.connector.connect(user='root', password='123456', host = 'chatbot_db', database='user')
            cursor = conn.cursor()

            # 下 query指令 範例：只找 mcolor_label = 2 的資料
            query = "SELECT shoesIDX, shoesID FROM `shoes` where shoesID = "+repr(product_id)
            cursor.execute(query)

            # 顯示讀取到的資料
            for shoesIDX, shoesID in cursor:
                product_id = int(shoesIDX)

            print('total', cursor.rowcount, 'shoes')
        # 關閉連線
        finally:
            cursor.close()

        rating_data = {"userid": userid, "shoes_id": product_id, "rating": rating}
        future = producer.send(topic="shoes_rating", value=rating_data)
        record_metadata = future.get(timeout=10)
        print(record_metadata)
        print("Message sending completed!")

    except Exception as e:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))


    finally:
        # 關掉Producer實例的連線
        producer.close()

In [9]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

現有三個欄位
menu, folder, tag

若folder欄位有值，則
    讀取其reply.json，轉譯成消息，並發送

若menu欄位有值，則
    讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
    讀取其reply.json，轉譯成消息，並發送

'''
from linebot.models import (
    PostbackEvent
)

from datetime import datetime

from urllib.parse import parse_qs, parse_qsl

from kafka import KafkaProducer

import random

count = 0

@handler.add(PostbackEvent)
def process_postback_event(event):
#     query_string_dict = parse_qsl(event.postback.data)
    query_string_dict = dict(parse_qsl(event.postback.data))
       
    print(query_string_dict)
#     if 'folder' in query_string_dict and query_string_dict.get('folder')[0] in os.listdir('./material/'):
    
#         result_message_array =[]

#         replyJsonPath = 'material/'+query_string_dict.get('folder')[0]+"/reply.json"
#         result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
#         line_bot_api.reply_message(
#             event.reply_token,
#             result_message_array
#         )
    if 'menu' in query_string_dict:
 
        linkRichMenuId = open("material/"+query_string_dict.get('menu')+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        replyJsonPath = 'material/'+query_string_dict.get('menu')+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    elif 'tag' in query_string_dict:
        print(query_string_dict.get('tag'))
        if query_string_dict.get('tag') == 'Productsrecommend':
            rating = query_string_dict.get('rating')
            product_id = query_string_dict.get('product')
            userid = query_string_dict.get('userid')
            userid = 611
#             userid = random.randint(1, 610)
            print(rating, product_id, userid)
            global count
            print(count)
            if count < 3:
               
                result_message_array = generate_flextemplate_dict(event.source.user_id)
                ##發送訊息
                line_bot_api.reply_message(
                    event.reply_token,
                    result_message_array
                )
                count += 1
                if count > 1:
                    sendkafka(rating, product_id, userid)
            else:
                count = 0
                result_message_array = findRecommend(userid)
                line_bot_api.reply_message(
                    event.reply_token,
                    result_message_array
#                     TextMessage(
#                     text='感謝您的回覆資訊，將提供專屬於您的個人推薦清單')
                )
        
        
         # 將用戶資訊存在檔案內
        with open("./users_event.txt", "a") as myfile:
            users_event = {
                'user_id': event.source.user_id,
                'tag': query_string_dict.get('tag'),
                'timestamp': event.timestamp
            }
            myfile.write(json.dumps(users_event,sort_keys=True))
            myfile.write('\r\n')
            
    elif 'rating' in query_string_dict:
#         producer = KafkaProducer(
#             # 指定Kafka集群伺服器(["docker.compose.host.name:9092"])
#             bootstrap_servers = ["kafka:9092"],
#             value_serializer = lambda m: json.dumps(m).encode('ascii'),
#         )
        rating = query_string_dict['rating']
        product_id = query_string_dict['product']
        userid = query_string_dict['userid']
        userid = random.randint(1, 610)
        print(rating, product_id, userid)
        sendkafka(rating, product_id, userid)
#         try:
#             import mysql.connector

#             cursor = None
#             conn = None
#             try:
#                 # 建立連線 並指定 database
#                 conn = mysql.connector.connect(user='root', password='123456', host = 'chatbot_db', database='user')
#                 cursor = conn.cursor()

#                 # 下 query指令 範例：只找 mcolor_label = 2 的資料
#                 query = "SELECT shoesIDX, shoesID FROM `shoes` where shoesID = "+repr(product_id)
#                 cursor.execute(query)

#                 # 顯示讀取到的資料
#                 for shoesIDX, shoesID in cursor:
#                     product_id = int(shoesIDX)

#                 print('total', cursor.rowcount, 'shoes')
#             # 關閉連線
#             finally:
#                 cursor.close()
            
#             rating_data = {"userid": userid, "shoes_id": product_id, "rating": rating}
#             future = producer.send(topic="shoes_rating", value=rating_data)
#             record_metadata = future.get(timeout=10)
#             print(record_metadata)
#             print("Message sending completed!")
            
#         except Exception as e:
#             # 錯誤處理
#             e_type, e_value, e_traceback = sys.exc_info()
#             print("type ==> %s" % (e_type))
#             print("value ==> %s" % (e_value))
#             print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
#             print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
#             print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))


#         finally:
#             # 關掉Producer實例的連線
#             producer.close()

In [10]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。
擷取主顏色

'''
#!pip install clarifai
# 引用套件
from linebot.models import ImageMessage
from clarifai.rest import Image as ClImage
from clarifai.rest import ClarifaiApp
import os
# from darknet_test import test_pic
import time


import warnings
warnings.filterwarnings('ignore')
import numpy as np
# import tensorflow
from keras.models import load_model
from keras.preprocessing.image import load_img
# 一定要使用那個模型的preprocess
from keras.applications.vgg19 import preprocess_input

cat_model = load_model('./models/cnn_newcolor_vgg19_20191030.h5')
imgMsgkey = None

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    global imgMsgkey
    if imgMsgkey == '圖片找鞋':
        message_content = line_bot_api.get_message_content(event.message.id)

        with open('./images/'+event.message.id+'.jpg', 'wb') as fd:
            for chunk in message_content.iter_content():
                fd.write(chunk)

        #同時存在一個，上限五則訊息
    #     line_bot_api.reply_message(
    #         event.reply_token,
    #         [TextSendMessage(text='Image has Upload'+ ' ' + event.message.id),
    #          TextSendMessage(text='等待30秒後，請稍後再做確認')
    #         ]
    #     )        


        #路徑為相對路徑
        Image_shoes = './images/'+event.message.id+'.jpg'

        pwd = '/root/darknet/result_img/'
        for img in [i for i in os.listdir(pwd) if '.jpg' in i]:
            filepath = pwd + img
            if os.path.exists(filepath):
                os.remove(filepath)
        pwd = '/root/linechatbot/material/'
        for img in [i for i in os.listdir(pwd) if '.jpg' in i]:
            filepath = pwd + img
            if os.path.exists(filepath):
                os.remove(filepath)

        # 送入darknet檢測照片
        CopyFileWaitToTest(Image_shoes)
        test_pic()
        os.chdir("/root/linechatbot")

        t0 = time.time()
        pwd = '/root/darknet/result_img'
        imgWaitToTest = [i for i in os.listdir(pwd) if '.jpg' in i]
        while len(imgWaitToTest) == 0:
            time.sleep(0.5)
            # 讀取資料夾內.jpg檔案的數量
            imgWaitToTest = [i for i in os.listdir(pwd) if '.jpg' in i]
            # 如果測試照片已經輸出，則複製照片到對外IP的資料夾內，等待發送給使用者
            if len(imgWaitToTest) != 0 or time.time() - t0 > 5:
                shoeimglist = CopyFileWaitToPost(event.source.user_id, event.message.id)
                time.sleep(0.5)
                break
            else:
                continue


        # 準備輸出多張照片的list
        result_message_array = []
        while True:
            pwd = '/root/linechatbot/material/user/'+event.source.user_id
            if os.path.exists(pwd):
                imgWaitToPost = [i for i in os.listdir(pwd) if '.jpg' in i]
            if len(imgWaitToPost) != 0 and len(shoeimglist) != 0:
                for img in imgWaitToPost:
                    url = 'https://'+server_url+'/material/user/'+event.source.user_id+'/'+img
                    print(url)
                    String = """ImageSendMessage(
                                 original_content_url="{}",
                                 preview_image_url="{}"
                             )""".format(url, url)
                    result_message_array.append(eval(String))
                line_bot_api.reply_message(
                    event.reply_token,
                    result_message_array
                )
                break
            else:
                line_bot_api.reply_message(
                    event.reply_token,
                    TextSendMessage(text='處理中，請稍後再確認結果，謝謝。')
                )
                break


        '''
            # ======================= 官方固定格式 ===============================
            app_color = ClarifaiApp(api_key='0f82436fcad44c1cb8953ee52d563ef1')
            # model = app.public_models.general_model
            model = app_color.models.get('color')
            # picnames = pic_list('D:\clarifai\pictures')
            image = ClImage(filename='/root/darknet/result_img/'+imgWaitToGetColor[-1])
            color_list = model.predict([image])["outputs"][0]["data"]["colors"]
            # ===================================================================
        #     print(color_list)
            color_datas = {}
            for color in color_list:
                color_id = color["raw_hex"]
                color_pa = color["value"]
                color_name = color["w3c"]["name"]
        #         print("color_data= ",  color_name, color_id + "\t", str(color_pa * 100)[:5] + "%")
                color_datas[color_pa]='main color = '+str(color_name) + " "+ str(color_id) + " " + str(color_pa * 100)[:5] + "%"
        #     print(color_datas)
        #     print(color_datas.keys())
            max_data = max(color_datas.keys())
            color_data = color_datas[max_data]
            print(color_data)

        '''
    elif imgMsgkey == '顏色找鞋':

        message_content = line_bot_api.get_message_content(event.message.id)
        
        filepath = './images/'+event.source.user_id+'/Colortofindshoes/'
        
        if not os.path.exists(filepath):
            os.makedirs(filepath)
        
        fn = filepath+event.message.id+'.jpg'
        
        with open(fn, 'wb') as fd:
            for chunk in message_content.iter_content():
                fd.write(chunk)
        from keras.models import load_model
        img = load_img(fn, target_size=(224,224)).convert("RGB")
        img_np = np.array(img)
        img_pre = preprocess_input(img_np)
        preimgs = np.array([img_pre])
        pre = cat_model.predict(preimgs)
        trans = ["black", "blue", "brown", "green", "red", "white", "yellow"]
        idx = pre.argmax(axis=1)[0]
        l = trans[idx]
        print("我覺得是", l)
        result_message_array = findColor2(l, event.source.user_id)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )

Using TensorFlow backend.


In [11]:
'''

準備QuickReply的Button


'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

# 創建QuickReplyButton 

## 點擊後，以用戶身份發送文字消息
## MessageAction
textQuickReplyButton = QuickReplyButton(
    action=MessageAction(
        label="確認結果", 
        text="確認結果"
    )
)

textQuickReplyButton2 = QuickReplyButton(
    action=MessageAction(
        label="確認顏色結果", 
        text="確認顏色結果"
    )
)

## 點擊後，彈跳出選擇時間之視窗
## DatetimePickerAction
dateQuickReplyButton = QuickReplyButton(
    action=DatetimePickerAction(
        label="時間選擇",
        data="data3",                       
        mode="date"
    )
)


## 點擊後，開啟相機
## CameraAction
cameraQuickReplyButton = QuickReplyButton(
    action=CameraAction(label="拍照")
)

## 點擊後，切換至照片相簿選擇
cameraRollQRB = QuickReplyButton(
    action=CameraRollAction(label="選擇照片")
)

## 點擊後，跳出地理位置
locationQRB = QuickReplyButton(
    action=LocationAction(label="地理位置")
)

## 點擊後，以Postback事件回應Server 
postbackQRB = QuickReplyButton(
    action=PostbackAction(label="我是PostbackEvent", data="data1")
)

In [12]:
'''

以QuickReply封裝該些QuickReply Button

'''
## 設計QuickReplyButton的List
quickReplyList = QuickReply(
    items = [textQuickReplyButton, cameraQuickReplyButton, cameraRollQRB, ]
)
quickReplyListColor = QuickReply(
    items = [cameraQuickReplyButton, cameraRollQRB, ]
)

In [13]:
'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
from linebot.models import (
    TextSendMessage,
)
# QuickReply的觸發點在於 Server傳送文字給用戶的時候
quickReplyTextSendMessage = TextSendMessage(text='請選擇圖片上傳方式或確認結果', quick_reply=quickReplyList)
quickReplyTextSendMessageColor = TextSendMessage(text='請選擇圖片上傳方式或確認結果', quick_reply=quickReplyListColor)

In [14]:
'''
設計一個字典

'''
template_message_dict = {
    "圖片找鞋":quickReplyTextSendMessage,
    "顏色找鞋":quickReplyTextSendMessageColor
}

In [15]:
from clarifai.rest import Image as ClImage
from clarifai.rest import ClarifaiApp

def GetColor(filePath):
    # ======================= 官方固定格式 ===============================
    app_color = ClarifaiApp(api_key='0f82436fcad44c1cb8953ee52d563ef1')
    # model = app.public_models.general_model
    model = app_color.models.get('color')
    # picnames = pic_list('D:\clarifai\pictures')
    image = ClImage(filename=filePath)
    color_list = model.predict([image])["outputs"][0]["data"]["colors"]
    # ===================================================================
#     print(color_list)
    color_datas = {}
    for color in color_list:
        color_id = color["raw_hex"]
        color_pa = color["value"]
        color_name = color["w3c"]["name"]
#         print("color_data= ",  color_name, color_id + "\t", str(color_pa * 100)[:5] + "%")
        color_datas[color_pa]='main color = '+str(color_name) + " "+ str(color_id) + " " + str(color_pa * 100)[:5] + "%"
#     print(color_datas)
#     print(color_datas.keys())
    max_data = max(color_datas.keys())
    color_data = color_datas[max_data]
    print(color_data)
    return color_data

In [16]:
import os
import shutil

def CopyFileWaitToTest(Image_shoes):
    Image_shoes = Image_shoes.split("/")[-1]
    Image_shoes = "/root/linechatbot/images/" + Image_shoes
    des_pwd = '/root/darknet/data/'
    fn = 'test.jpg'
    shutil.copyfile(Image_shoes, des_pwd+fn)

# CopyFileWaitToTest('./image/10744146872237.jpg')

In [17]:
import os

def test_pic():   
    os.chdir("/root/darknet")
    cmd ="""
        ./darknet detector test \
        shoe_person/cfg/obj.data \
        shoe_person/cfg/yolov3.cfg \
        shoe_person/cfg/weights/yolov3_last.weights -thresh 0.5\
        data/test.jpg \
        -dont_show &
        """
    os.popen(cmd)

In [18]:
import os
import shutil

def CopyFileWaitToPost(user_id, messageID):
    origin_pwd = '/root/darknet/result_img/'
    des_pwd = '/root/linechatbot/material/user/'+user_id+'/'
    if not os.path.exists(des_pwd):
        os.makedirs(des_pwd)
        
    # 清空原先測試照片
    for img in [i for i in os.listdir(des_pwd) if '.jpg' in i]:
        os.remove(des_pwd+img)
    img_list = os.listdir(origin_pwd)
    returnImgList = []
    for i in img_list:
        if 'shoe' in i:
            print(i)
            NewfileName = messageID + '-' + i
            shutil.copyfile(origin_pwd+i, des_pwd+NewfileName)
            returnImgList.append(NewfileName)
    return returnImgList

In [19]:
import mysql.connector

def findSituation(l):
    try:
        # 建立連線 並指定 database
        conn = mysql.connector.connect(user='root', password='123456', host = 'chatbot_db', database='TEST1')
        cursor = conn.cursor()

        # 下 query指令 範例：只找 pid, pro_name, brand, url 的資料
        query = "SELECT pid, pro_name, brand FROM shoes_data WHERE cata_label = "+repr(l)+"ORDER BY RAND() LIMIT 4"
        cursor.execute(query)

        # 儲存讀取到的資料
        col_list = []
        for pid, pro_name, brand in cursor:
            pic_url = 'https://'+server_url+'/material/product/'+pid+'.jpg'
            print(pid, pro_name, brand, pic_url)
            #循環加到 list_col_color
            col = {
                    "thumbnailImageUrl": pic_url,
                    "title": pro_name[:40],
                    "text": "所顏是否?",
                    "actions": [
                      {
                        "type": "postback",
                        "label": "喜歡",
                        "text": "加入喜好清單",
                        "data": "tag=maincolor|Secondarycolor|"+pid
                      },
                      {
                        "type": "postback",
                        "label": "不喜歡",
                        "text": "不喜歡"+pro_name,
                        "data": "tag=maincolor|Secondarycolor|"+pid
                      }
                    ]
                  }
            col_list.append(col)

        #輸出到linebot輪播
        data = [
            {
                "type": "template",
                "altText": "情境輸入推薦清單",
                "template":{
                        "type": "carousel",
                        "actions": [],
                        "columns": col_list
                }
            }
        ]

        result_message_array = detect_json_array_to_linebot_message_array(data)
    

    # 關閉連線
    finally:
        cursor.close()
        return result_message_array

In [20]:
import mysql.connector

def findSituation2(l, userid):
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(userid)
    try:
        # 建立連線 並指定 database
        conn = mysql.connector.connect(user='root', password='123456', host = 'chatbot_db', database='TEST1')
        cursor = conn.cursor()

        # 下 query指令 範例：只找 pid, pro_name, brand, url 的資料
        query = "SELECT pid, pro_name, brand FROM shoes_data WHERE cata_label = "+repr(l)+"ORDER BY RAND() LIMIT 4"
        cursor.execute(query)
        
                # 顯示讀取到的資料
        contentsList = []
        for pid, pro_name, brand in cursor:
            pic_url = 'https://'+server_url+'/material/product/'+pid+'.jpg'
            print(pid, pro_name, brand, pic_url)
            #循環加到 list_col_color

            bubbleDict = {
                    "type": "bubble",
                    "direction": "ltr",
                    "hero": {
                      "type": "image",
                      "url": pic_url,
                      "size": "full",
                      "aspectRatio": "1.51:1",
                      "aspectMode": "fit"
                    },
                    "body": {
                      "type": "box",
                      "layout": "vertical",
                      "contents": [
                        {
                          "type": "text",
                          "text": brand,
                          "align": "center",
                          "weight": "bold"
                        },
                        {
                          "type": "text",
                          "text": pro_name[:40],
                          "align": "center"
                        },
                        {
                          "type": "box",
                          "layout": "horizontal",
                          "contents": [
                            {
                              "type": "button",
                              "action": {
                                "type": "postback",
                                "label": "加入購物車",
                                "text": "我想買"+pro_name[:10],
                                "data": "tag=cart&product="+pid+"&userid="+userid
                              }
                            },
                            {
                              "type": "button",
                              "action": {
                                "type": "postback",
                                "label": "加入收藏",
                                "text": pro_name[:10]+"加入收藏",
                                "data": "tag=collection&product="+pid+"&userid="+userid
                              }
                            }
                          ]
                        },
                        {
                          "type": "separator"
                        }
                      ]
                    },
                    "footer": {
                      "type": "box",
                      "layout": "horizontal",
                      "contents": [
                        {
                          "type": "text",
                          "text": "評分",
                          "gravity": "center"
                        },
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "1",
                            "data": "rating=1&product="+pid+"&userid="+userid
                          }
                        },
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "2",
                            "data": "rating=2&product="+pid+"&userid="+userid
                          }
                        },
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "3",
                            "data": "rating=3&product="+pid+"&userid="+userid
                          }
                        },
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "4",
                            "data": "rating=4&product="+pid+"&userid="+userid
                          }
                        },
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "5",
                            "data": "rating=5&product="+pid+"&userid="+userid
                          }
                        }
                      ]
                    }
                  }

            contentsList.append(bubbleDict)
        Flex_Message = [
            {
              "type": "text",
              "text": str(user_profile.display_name) + "您好\n根據您的情境，可能適合以下鞋款"+u"\U0001f600"
            },
            {
              "type": "flex",
              "altText": "情境輸入推薦清單",
              "contents": {
                "type": "carousel",
                "contents": contentsList
              }
            }]
        result_message_array = detect_json_array_to_linebot_message_array(Flex_Message)
    # 關閉連線
    finally:
        cursor.close()
        return result_message_array

In [21]:
import mysql.connector

secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")


def findColor(l):
    try:
        # 建立連線 並指定 database
        conn = mysql.connector.connect(user='root', password='123456', host = 'chatbot_db', database='TEST1')
        cursor = conn.cursor()

        # 下 query指令 範例：只找 mcolor_label = 2 的資料
        query = "SELECT pid, pro_name, brand FROM shoes_data WHERE coler_label = "+repr(l)+" && mcolor_name like '%"+l+"%' ORDER BY RAND(), mcolor_pa desc LIMIT 4"
        cursor.execute(query)

        # 顯示讀取到的資料
        col_list = []
        for pid, pro_name, brand in cursor:
            pic_url = 'https://'+server_url+'/material/product/'+pid+'.jpg'
            print(pid, pro_name, brand, pic_url)
            #循環加到 list_col_color
            col = {
                    "thumbnailImageUrl": pic_url,
                    "title": pro_name[:40],
                    "text": "所顏是否?",
                    "actions": [
                      {
                        "type": "postback",
                        "label": "喜歡",
                        "text": "加入喜好清單",
                        "data": "tag=maincolor|Secondarycolor|"+pid
                      },
                      {
                        "type": "postback",
                        "label": "不喜歡",
                        "text": "不喜歡"+pro_name,
                        "data": "tag=maincolor|Secondarycolor|"+pid
                      }
                    ]
                  }
            col_list.append(col)

        #輸出到linebot輪播
        data = [
            {
                "type": "template",
                "altText": "顏色找鞋推薦清單",
                "template":{
                        "type": "carousel",
                        "actions": [],
                        "columns": col_list
                }
            }
        ]
        result_message_array = detect_json_array_to_linebot_message_array(data)
    # 關閉連線
    finally:
        cursor.close()
        return result_message_array

In [22]:
import mysql.connector
import json



def findColor2(l, userid):
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(userid)
    try:
        # 建立連線 並指定 database
        conn = mysql.connector.connect(user='root', password='123456', host = 'chatbot_db', database='TEST1')
        cursor = conn.cursor()

        # 下 query指令 範例：只找 mcolor_label = 2 的資料
        query = "SELECT pid, pro_name, brand FROM shoes_data WHERE coler_label = "+repr(l)+" && mcolor_name like '%"+l+"%' ORDER BY RAND(), mcolor_pa desc LIMIT 5"
        cursor.execute(query)

        # 顯示讀取到的資料
        contentsList = []
        for pid, pro_name, brand in cursor:
            pic_url = 'https://'+server_url+'/material/product/'+pid+'.jpg'
            print(pid, pro_name, brand, pic_url)
            #循環加到 list_col_color

            bubbleDict = {
                    "type": "bubble",
                    "direction": "ltr",
                    "hero": {
                      "type": "image",
                      "url": pic_url,
                      "size": "full",
                      "aspectRatio": "1.51:1",
                      "aspectMode": "fit"
                    },
                    "body": {
                      "type": "box",
                      "layout": "vertical",
                      "contents": [
                        {
                          "type": "text",
                          "text": brand,
                          "align": "center",
                          "weight": "bold"
                        },
                        {
                          "type": "text",
                          "text": pro_name[:40],
                          "align": "center"
                        },
                        {
                          "type": "box",
                          "layout": "horizontal",
                          "contents": [
                            {
                              "type": "button",
                              "action": {
                                "type": "postback",
                                "label": "加入購物車",
                                "text": "我想買"+pro_name[:10],
                                "data": "tag=cart&product="+pid+"&userid="+userid
                              }
                            },
                            {
                              "type": "button",
                              "action": {
                                "type": "postback",
                                "label": "加入收藏",
                                "text": pro_name[:10]+"加入收藏",
                                "data": "tag=collection&product="+pid+"&userid="+userid
                              }
                            }
                          ]
                        },
                        {
                          "type": "separator"
                        }
                      ]
                    },
                    "footer": {
                      "type": "box",
                      "layout": "horizontal",
                      "contents": [
                        {
                          "type": "text",
                          "text": "評分",
                          "gravity": "center"
                        },
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "1",
                            "data": "rating=1&product="+pid+"&userid="+userid
                          }
                        },
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "2",
                            "data": "rating=2&product="+pid+"&userid="+userid
                          }
                        },
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "3",
                            "data": "rating=3&product="+pid+"&userid="+userid
                          }
                        },
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "4",
                            "data": "rating=4&product="+pid+"&userid="+userid
                          }
                        },
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "5",
                            "data": "rating=5&product="+pid+"&userid="+userid
                          }
                        }
                      ]
                    }
                  }

            contentsList.append(bubbleDict)
        Flex_Message = [
            {
              "type": "text",
              "text": str(user_profile.display_name) + "您好\n以下是專屬於您的顏色找鞋商品推薦"+u"\U0001f600"+"\n"
            },
            {
              "type": "flex",
              "altText": "顏色找鞋推薦清單",
              "contents": {
                "type": "carousel",
                "contents": contentsList
              }
            }]
        result_message_array = detect_json_array_to_linebot_message_array(Flex_Message)
    # 關閉連線
    finally:
        cursor.close()
        return result_message_array

In [23]:
"""
建立flextemplate
"""


def generate_flextemplate_dict(userid):
    try:
        # 建立連線 並指定 database
        conn = mysql.connector.connect(user='root', password='123456', host = 'chatbot_db', database='TEST1')
        cursor = conn.cursor()

        # 下 query指令 範例：只找 mcolor_label = 2 的資料
        query = "SELECT pid, pro_name, brand FROM shoes_data ORDER BY RAND() LIMIT 1"
        cursor.execute(query)

        # 顯示讀取到的資料
    #     for pid, mcolor_label, scolor_label, mcolor_id in cursor:
    #         print('pid={}, m_label={}, s_label={}, mc_id={}'.format(pid, mcolor_label, scolor_label, mcolor_id))
        contentsList = []
        for pid, pro_name, brand in cursor:
            pic_url = 'https://'+server_url+'/material/product/'+pid+'.jpg'
            print(pid, pro_name, brand, pic_url)
            #循環加到 list_col_color
            Flex_Message = [{
                          "type": "flex",
                          "altText": "Flex Message",
                          "contents": {
                            "type": "bubble",
                            "direction": "ltr",
                            "header": {
                              "type": "box",
                              "layout": "vertical",
                              "contents": [
                                {
                                  "type": "text",
                                  "text": "請您對這雙鞋作評價",
                                  "align": "center",
                                  "weight": "bold"
                                }
                              ]
                            },
                            "hero": {
                              "type": "image",
                              "url": pic_url,
                              "size": "full",
                              "aspectRatio": "1.51:1",
                              "aspectMode": "fit"
                            },
                            "body": {
                              "type": "box",
                              "layout": "vertical",
                              "contents": [
                                {
                                  "type": "text",
                                  "text": brand,
                                  "align": "center",
                                  "weight": "bold"
                                },
                                {
                                  "type": "text",
                                  "text": pro_name,
                                  "align": "center"
                                },
                                {
                                  "type": "box",
                                  "layout": "horizontal",
                                  "contents": [
                                    {
                                      "type": "button",
                                      "action": {
                                        "type": "postback",
                                        "label": "加入購物車",
                                        "text": "::我要:"+pro_name[:10],
                                        "data": "tag=cart&product="+pid+"&userid="+userid
                                      }
                                    },
                                    {
                                      "type": "button",
                                      "action": {
                                        "type": "postback",
                                        "label": "加入收藏",
                                        "text": pro_name[:10]+"::加入收藏:",
                                        "data": "tag=collection&product="+pid+"&userid="+userid
                                      }
                                    }
                                  ]
                                },
                                {
                                  "type": "separator"
                                }
                              ]
                            },
                            "footer": {
                              "type": "box",
                              "layout": "horizontal",
                              "contents": [
                                {
                                  "type": "text",
                                  "text": "評分",
                                  "gravity": "center"
                                },
                                {
                                  "type": "button",
                                  "action": {
                                    "type": "postback",
                                    "label": "1",
                                    "data": "tag=Productsrecommend&rating=1&product="+pid+"&userid="+userid
                                  }
                                },
                                {
                                  "type": "button",
                                  "action": {
                                    "type": "postback",
                                    "label": "2",
                                    "data": "tag=Productsrecommend&rating=2&product="+pid+"&userid="+userid
                                  }
                                },
                                {
                                  "type": "button",
                                  "action": {
                                    "type": "postback",
                                    "label": "3",
                                    "data": "tag=Productsrecommend&rating=3&product="+pid+"&userid="+userid
                                  }
                                },
                                {
                                  "type": "button",
                                  "action": {
                                    "type": "postback",
                                    "label": "4",
                                    "data": "tag=Productsrecommend&rating=4&product="+pid+"&userid="+userid
                                  }
                                },
                                {
                                  "type": "button",
                                  "action": {
                                    "type": "postback",
                                    "label": "5",
                                    "data": "tag=Productsrecommend&rating=5&product="+pid+"&userid="+userid
                                  }
                                }
                              ]
                            }
                          }
                        }]
        result_message_array = detect_json_array_to_linebot_message_array(Flex_Message)

    # 關閉連線
    finally:
        cursor.close()
        return result_message_array

In [24]:
# 找出使用者推薦清單的輪播訊訊息陣列

def findRecommend(userid):
    # 取出消息內User的資料
#     user_profile = line_bot_api.get_profile(userid)
    # 取得user的推薦清單
    RecommendIDList = getRecommendList(userid)
    RecommendDataList = getRecommendDataList(RecommendIDList)
    
    
    # 準備放入輪播清單的list
    contentsList = []
    for pid, pro_name, brand in RecommendDataList:
        pic_url = 'https://'+server_url+'/material/product/'+pid+'.jpg'
        print(pid, pro_name, brand, pic_url)
        #循環加到 list_col_color

        bubbleDict = {
                "type": "bubble",
                "direction": "ltr",
                "hero": {
                  "type": "image",
                  "url": pic_url,
                  "size": "full",
                  "aspectRatio": "1.51:1",
                  "aspectMode": "fit"
                },
                "body": {
                  "type": "box",
                  "layout": "vertical",
                  "contents": [
                    {
                      "type": "text",
                      "text": brand,
                      "align": "center",
                      "weight": "bold"
                    },
                    {
                      "type": "text",
                      "text": pro_name[:40],
                      "align": "center"
                    },
                    {
                      "type": "box",
                      "layout": "horizontal",
                      "contents": [
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "加入購物車",
                            "text": "我想買"+pro_name[:10],
                            "data": "tag=cart&product="+pid+"&userid="+str(userid)
                          }
                        },
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "加入收藏",
                            "text": pro_name[:10]+"加入收藏",
                            "data": "tag=collection&product="+pid+"&userid="+str(userid)
                          }
                        }
                      ]
                    },
                    {
                      "type": "separator"
                    }
                  ]
                },
                "footer": {
                  "type": "box",
                  "layout": "horizontal",
                  "contents": [
                    {
                      "type": "text",
                      "text": "評分",
                      "gravity": "center"
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "1",
                        "data": "rating=1&product="+pid+"&userid="+str(userid)
                      }
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "2",
                        "data": "rating=2&product="+pid+"&userid="+str(userid)
                      }
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "3",
                        "data": "rating=3&product="+pid+"&userid="+str(userid)
                      }
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "4",
                        "data": "rating=4&product="+pid+"&userid="+str(userid)
                      }
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "5",
                        "data": "rating=5&product="+pid+"&userid="+str(userid)
                      }
                    }
                  ]
                }
              }

        contentsList.append(bubbleDict)
    Flex_Message = [
        {
          "type": "text",
          "text": "以下是根據您看過的商品，\n你可能喜歡以下商品"+u"\U0001f600"
        },
        {
          "type": "flex",
          "altText": "商品推薦",
          "contents": {
            "type": "carousel",
            "contents": contentsList
          }
        }]
    result_message_array = detect_json_array_to_linebot_message_array(Flex_Message)
    return result_message_array

# findRecommend(611)

In [25]:
import redis

def getRecommendList(userid):
    conn3 = redis.StrictRedis(
        host='redis',
        port="6379",
        password='123456',
        ssl=False, charset="utf-8", decode_responses=True)
    shoes_list = eval(conn3.hget("shoes", userid))
    
    recommendIDlist = []
    for pid, _, _ in shoes_list:
        recommendIDlist.append(pid)
    
    return recommendIDlist

In [26]:
import mysql.connector

def getRecommendDataList(RecommendIDList, num=5):
    RecommendDataList = []
    for i, s in enumerate(RecommendIDList):
        if i > num:
            break
        pid = s
        cursor = None
        conn = None
        try:
            # 建立連線 並指定 database
            conn = mysql.connector.connect(user='root', password='123456', host = 'chatbot_db', database='TEST1')
            cursor = conn.cursor()

            # 下 query指令 範例：只找 mcolor_label = 2 的資料
            query = "SELECT pid, pro_name, brand FROM `shoes_data` where pid ="+repr(pid)
            cursor.execute(query)
            
            for c in cursor:
                RecommendDataList.append(c)

        # 關閉連線
        finally:
            cursor.close()
    return RecommendDataList


In [27]:
# recommend_list = [{'pid': 'CI2082-800',
#   'pro_name': 'Nike Air Zoom Pegasus 36 (Oklahoma State)',
#   'brand': 'nike',
#   'pic_url': 'https://c.static-nike.com/a/images/t_PDP_1280_v1/f_auto/b10nnuzkw7iodqzstdct/air-zoom-pegasus-36-texas-mens-running-shoe-r5Rd7m.jpg',
#   'price': '130.0'},
#  {'pid': '66230_CDB',
#   'pro_name': "Men's Moreno - Pence",
#   'brand': 'skechers',
#   'pic_url': 'https://image.skechers.com/img/productimages/xlarge/66230_CDB_E.jpg',
#   'price': '80.0'},
#  {'pid': '5526882327952111',
#   'pro_name': 'Alperton 2.0 Leather Boots',
#   'brand': 'belstaff',
#   'pic_url': 'https://cache.mrporter.com/variants/images/5526882327952111/in/w2000_q80.jpg',
#   'price': '575'},
#  {'pid': '1193A055-600',
#   'pro_name': 'GEL-Mai Knit MT',
#   'brand': 'asics',
#   'pic_url': 'https://asics.scene7.com/is/image/asics/1193A055_600_0020000513_RT?$dw_temp_default_gallery$&wid=1280&hei=640&extend=10,10,10,10&qlt=100',
#   'price': '79.99'},
#  {'pid': '16635_CHOC',
#   'pro_name': "Women's Skechers On the GO Joy - Breezie",
#   'brand': 'skechers',
#   'pic_url': 'https://image.skechers.com/img/productimages/xlarge/16635_CHOC_E.jpg',
#   'price': '65.0'}]

In [28]:
# 找出使用者推薦清單的輪播訊訊息陣列

def Picfindshoe(userid, recommend_list):
    # 取出消息內User的資料
#     user_profile = line_bot_api.get_profile(userid)  
    
    # 準備放入輪播清單的list
    contentsList = []
    for r in recommend_list:
#         print(r["pid"], r["pro_name"], r["brand"], r["price"])
        pid = r["pid"]
        pro_name = r["pro_name"]
        brand = r["brand"]
        price = r["price"]
        pic_url = 'https://'+server_url+'/material/product/'+pid+'.jpg'
#         print(pid, pro_name, brand, pic_url)
        #循環加到 list_col_color

        bubbleDict = {
                "type": "bubble",
                "direction": "ltr",
                "hero": {
                  "type": "image",
                  "url": pic_url,
                  "size": "full",
                  "aspectRatio": "1.51:1",
                  "aspectMode": "fit"
                },
                "body": {
                  "type": "box",
                  "layout": "vertical",
                  "contents": [
                    {
                      "type": "text",
                      "text": brand,
                      "align": "center",
                      "weight": "bold"
                    },
                    {
                      "type": "text",
                      "text": pro_name[:40],
                      "align": "center"
                    },
                    {
                      "type": "text",
                      "text": price,
                      "align": "center"
                    },
                    {
                      "type": "box",
                      "layout": "horizontal",
                      "contents": [
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "加入購物車",
                            "text": "我想買"+pro_name[:10],
                            "data": "tag=cart&product="+pid+"&userid="+str(userid)
                          }
                        },
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "加入收藏",
                            "text": pro_name[:10]+"加入收藏",
                            "data": "tag=collection&product="+pid+"&userid="+str(userid)
                          }
                        }
                      ]
                    },
                    {
                      "type": "separator"
                    }
                  ]
                },
                "footer": {
                  "type": "box",
                  "layout": "horizontal",
                  "contents": [
                    {
                      "type": "text",
                      "text": "評分",
                      "gravity": "center"
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "1",
                        "data": "rating=1&product="+pid+"&userid="+str(userid)
                      }
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "2",
                        "data": "rating=2&product="+pid+"&userid="+str(userid)
                      }
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "3",
                        "data": "rating=3&product="+pid+"&userid="+str(userid)
                      }
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "4",
                        "data": "rating=4&product="+pid+"&userid="+str(userid)
                      }
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "5",
                        "data": "rating=5&product="+pid+"&userid="+str(userid)
                      }
                    }
                  ]
                }
              }

        contentsList.append(bubbleDict)
    Flex_Message = [
        {
          "type": "text",
          "text": "您好\n以下是根據您的上傳的照片，有以下相似的商品"+u"\U0001f600"+"\n"
        },
        {
          "type": "flex",
          "altText": "圖片找鞋推薦",
          "contents": {
            "type": "carousel",
            "contents": contentsList
          }
        }]
    result_message_array = detect_json_array_to_linebot_message_array(Flex_Message)
    return result_message_array

# result_message_array = Picfindshoe(611, recommend_list)
# result_message_array
# json.dumps(result_message_array, indent=2)

In [29]:
# ====  這些是removebg 和clarifai 套件  ========
import requests
from clarifai.rest import Image as ClImage
from clarifai.rest import ClarifaiApp
# =============================================

# =========  這是模型檔的套件  ===============
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
# ============================================

import mysql.connector
import pandas as pd 
import numpy as np
import os
import math

clf = joblib.load('./models/scol_model.pkl')

def remove_bg(testimg):


    # 連線removebg
    response = requests.post(
        'https://api.remove.bg/v1.0/removebg',
        files={'image_file': open(testimg, 'rb')},
        data={'size': 'auto'},
        headers={'X-Api-Key': 'ia8jDiTKSkupNnMsxwZ1e6um'},
    )
    # 輸出 去被 照片
    if response.status_code == requests.codes.ok:
        rm_test = testimg.split(".")[-2]+"-nobg.jpg"
        with open(testimg.split(".")[-2]+"-nobg.jpg", 'wb') as out:
            out.write(response.content)
    else:
        print("Error:", response.status_code, response.text)

# =====================   進入clarifai   ==========================

    # 給予金鑰
    app = ClarifaiApp(api_key='2ff7f19d333346cf88f7d6c9b382952c')

    # 選擇顏色model
    model = app.models.get('color')

    # 輸入圖片位置+檔名
    image = ClImage(filename= rm_test)

    color_list = model.predict([image])["outputs"][0]["data"]["colors"]
    

    major_color_list = []
    major_id_list = []
    major_pa_list = []
    second_color_list = []
    second_id_list = []
    second_pa_list = []

    color_dic = {
                "color_id": [],
                "color_name": [],
                "color_pa": []
                }

    for color in color_list:
        color_id = color["raw_hex"]
        color_pa = color["value"]
        color_name = color["w3c"]["name"]

        color_dic["color_id"].append(color_id)
        color_dic["color_name"].append(color_name)
        color_dic["color_pa"].append(color_pa)

#        print("color_data= ",  color_name, color_id + "\t", str(color_pa * 100)[:5] + "%")

    
# =================   依照顏色pa數大小重新排序   ======================================
    for i in range(len(color_dic["color_pa"]) - 1):
        for j in range(len(color_dic["color_pa"])-i-1):
            if color_dic["color_pa"][j] < color_dic["color_pa"][j+1]:
                color_dic["color_pa"][j], color_dic["color_pa"][j+1] = color_dic["color_pa"][j+1], color_dic["color_pa"][j]
                color_dic["color_name"][j], color_dic["color_name"][j+1] = color_dic["color_name"][j+1], color_dic["color_name"][j]
                color_dic["color_id"][j], color_dic["color_id"][j+1] = color_dic["color_id"][j+1], color_dic["color_id"][j]


# ==================   將主要色 append 到 major_color_list   ===========================

    major_color_list.append(color_dic["color_name"][0])
    major_id_list.append(color_dic["color_id"][0])
    major_pa_list.append(color_dic["color_pa"][0])
# =====================================================================================

# =================   將次要色 append 到 second_color_list  ===========================
    try:
        second_color_list.append(color_dic["color_name"][1])
        second_id_list.append(color_dic["color_id"][1])
        second_pa_list.append(color_dic["color_pa"][1])

# ================   若沒有第2種顏色 塞入主要顏色資料   =============================
    except:
        second_color_list.append(color_dic["color_name"][0])
        second_id_list.append(color_dic["color_id"][0])
        second_pa_list.append(color_dic["color_pa"][0])

# ====================================================================================

# =================   將其他色 append 到 other_color_list  ===========================
#    try:
#        other_color_list.append(color_dic["color_name"][2])
#        other_id_list.append(color_dic["color_id"][2])
#        other_pa_list.append(color_dic["color_pa"][2])
#    except:
#        other_color_list.append(color_dic["color_name"][0])
#        other_id_list.append(color_dic["color_id"][0])
#        other_pa_list.append(color_dic["color_pa"][0])

# ============   將要的顏色資訊取出來   ================
    mcol_name = major_color_list[0]
    mcol_id = major_id_list[0]
    mcol_pa = major_pa_list[0]

    scol_name = second_color_list[0]
    scol_id = second_id_list[0]
    scol_pa = second_pa_list[0]


# ================   對次要顏色取hsv   =====================
    # 轉10進制
    scol_rgb = tuple(int(scol_id[i:i+2], 16) for i in (1, 3, 5))
    scol_r = scol_rgb[0]
    scol_g = scol_rgb[1]
    scol_b = scol_rgb[2]
    
# =====================================================================
# 定義 rgb 轉 hsv 的 function

    def rgb_to_hsv(r, g, b):
        r, g, b = r/255.0, g/255.0, b/255.0
        mx = max(r, g, b)
        mn = min(r, g, b)
        df = mx-mn
        if mx == mn:
            h = 0
        elif mx == r:
            h = (60 * ((g-b)/df) + 360) % 360
        elif mx == g:
            h = (60 * ((b-r)/df) + 120) % 360
        elif mx == b:
            h = (60 * ((r-g)/df) + 240) % 360
        if mx == 0:
            s = 0
        else:
            s = (df/mx)*100
        v = mx*100
        return h, s, v

# ===================================================
# ===========   將hsv 正規化   ======================
    sh, ss, sv = rgb_to_hsv(scol_r, scol_g, scol_b)
    sh = math.cos(sh*math.pi/180)
    sh = (sh+1)/2
    ss = ss/100
    sv = sv / 100
# ===================================================
# ============   先找次要顏色標籤   ===========

    data = {'MH':[sh], 'MS':[ss], 'MV':[sv]} 
  
    # Create DataFrame 
    df_data = pd.DataFrame(data) 
# ================ !!!!!!!!!!!!!  模型名稱要改(clf)  !!!!!!!!!!!!!!!!!!! ===================
    scol_label = clf.predict(df_data)[0]
    scol_label = str(scol_label)

# ===============   得到 次要顏色標籤後 要將主要顏色資訊轉成 向量   ====================

    mcol_rgb_vec = list(tuple(int(mcol_id[i:i+2], 16) for i in (1, 3, 5)))
    mcol_rgb_vec.append(mcol_pa)
    mcol_rgb_vec = np.array(mcol_rgb_vec)

# ====================================================================================

# ====================   打開MySQL 尋找顏色相似資料   ===============================
    cursor = None
    conn = None

    try:
        # 建立連線 並指定 database
        conn = mysql.connector.connect(user='root', password='123456', host = 'chatbot_db', database='TEST1')
        cursor = conn.cursor()
    
        # 下 query指令 範例：只找 mcolor_label = 2 的資料
        query = """SELECT pid, pro_name, brand, pic_url, price, mcolor_label, scolor_label, mcolor_id, scolor_id, mcolor_pa, scolor_pa, mcolor_name, scolor_name
                    FROM shoes_data 
                    WHERE scolor_label = %s"""
      
        scolor_label = scol_label

    
        cursor.execute(query, (scolor_label, ))

        emps = cursor.fetchall()
    

        data = {'pid':[], 'pro_name':[], 'brand':[], 'pic_url':[], 'price':[], 'm_label':[], 's_label':[], 'mc_sim':[]} 


    
        for emp in emps:

            # 將次要顏色id 轉成rgb
            rgb_vec = list(tuple(int(emp[7][i:i+2], 16) for i in (1, 3, 5)))

            # append 顏色趴數
            rgb_vec.append(emp[9])
            rgb_vec = np.array(rgb_vec)
            # 歐式距離算顏色相似度
            dist = np.sqrt(np.sum(np.square(rgb_vec - mcol_rgb_vec)))
        
            data['mc_sim'].append(dist)
            data["pid"].append(emp[0])
            data["pro_name"].append(emp[1])
            data["brand"].append(emp[2])
            data["pic_url"].append(emp[3])
            data["price"].append(emp[4])
            data["m_label"].append(emp[5])
            data["s_label"].append(emp[6])
        
    
   
        df = pd.DataFrame(data)

# 關閉連線
    finally:
        cursor.close()

    df1 = df.sort_values(by=['mc_sim'],ascending=[True]).head(5)

    pid_list = list(df1["pid"])
    name_list = list(df1["pro_name"])
    brand_list = list(df1["brand"])
    pic_url_list = list(df1["pic_url"])
    price_list = list(df1["price"])

    recommend_list = []

    for i in range(len(df1["pid"])):
        pro_data = {
            "pid": pid_list[i],
            "pro_name": name_list[i],
            "brand": brand_list[i],
            "pic_url": pic_url_list[i],
            "price": price_list[i]
        }
        recommend_list.append(pro_data)

    return recommend_list


In [ ]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.18.0.10 - - [16/Nov/2019 05:16:37] "POST / HTTP/1.1" 200 -
172.18.0.10 - - [16/Nov/2019 05:16:38] "GET /material/關注/logo_push_removebg.png HTTP/1.1" 200 -
172.18.0.10 - - [16/Nov/2019 05:16:38] "GET /material/關注/logo_push_removebg.png HTTP/1.1" 200 -
172.18.0.10 - - [16/Nov/2019 05:16:38] "GET /material/關注/logo_push_removebg.png HTTP/1.1" 200 -


In [ ]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])